In [1]:
!pip install opentrons

  Obtaining dependency information for opentrons from https://files.pythonhosted.org/packages/55/23/95809f82a12176b9f4bcbbe5575f523f14721ac09d919c3a6802a7e3e96b/opentrons-7.5.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for opentrons-shared-data==7.5.0 from https://files.pythonhosted.org/packages/f0/a5/75ea901f0ff962ac35f74497813d5f0fa4c590029abbf4153a6c5e2779d8/opentrons_shared_data-7.5.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for aionotify==0.2.0 from https://files.pythonhosted.org/packages/d4/ca/104c3332c6c9be78108840fb3b08f40cfd2c598df4ce1cd7eb999416825c/aionotify-0.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<4.0.0,>=3.6.1 from https://files.pythonhosted.org/packages/19/24/44299477fe7dcc9cb58d0a57d5a7588d6af2ff403fdd2d47a246c91a3246/anyio-3.7.1-py3-none-any.whl.metadata
  Using cached anyio-3.7.1-py3-none-any.whl.metadata (4.7 kB)
  Obtaining dependency information for jsonschema<4.18.0,>=3.0.1 from http

In [5]:
from opentrons import simulate, protocol_api

# metadata
metadata = {
    'protocolName': 'PCR Setup',
    'author': 'Your Name <your.email@example.com>',
    'description': 'Add mastermix, forward primers, and reverse primers to destination plate',
    'apiLevel': '2.11'
}

def run(protocol: protocol_api.ProtocolContext):

    # labware
    forward_plate = protocol.load_labware('corning_96_wellplate_360ul_flat', '1')
    reverse_plate = protocol.load_labware('corning_96_wellplate_360ul_flat', '2')
    destination_plate = protocol.load_labware('corning_96_wellplate_360ul_flat', '3')
    mastermix_tube = protocol.load_labware('opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap', '4').wells_by_name()['A1']

    # pipettes
    p300 = protocol.load_instrument('p300_single_gen2', 'right', tip_racks=[protocol.load_labware('opentrons_96_tiprack_300ul', '5')])
    p10 = protocol.load_instrument('p10_single', 'left', tip_racks=[protocol.load_labware('opentrons_96_tiprack_10ul', '6')])

    # transfer 92µl of mastermix to each well in the destination plate
    for well in destination_plate.wells():
        p300.transfer(92, mastermix_tube, well, new_tip='never')

    # list of well names in 96-well format from A1 to G12
    wells_96 = [f'{row}{col}' for row in 'ABCDEFGH' for col in range(1, 13)]

    # transfer 4µl of forward and reverse primers to the destination plate
    for well in wells_96[:84]:  # from A1 to G12 (inclusive)
        p10.transfer(4, forward_plate[well], destination_plate[well], new_tip='always')
        p10.transfer(4, reverse_plate[well], destination_plate[well], new_tip='always')

# simulate the protocol
protocol_context = simulate.get_protocol_api('2.11')
run(protocol_context)

# print the protocol commands
for command in protocol_context.commands():
    print(command)


/Users/lucaslevassor/.opentrons/robot_settings.json not found. Loading defaults
Deck calibration not found.
/Users/lucaslevassor/.opentrons/deck_calibration.json not found. Loading defaults


OutOfTipsError: 

In [4]:
from opentrons import simulate
# simulate the protocol
protocol_context = simulate.get_protocol_api('2.11')
run(protocol_context)

# print the protocol commands
for command in protocol_context.commands():
    print(command)

/Users/lucaslevassor/.opentrons/robot_settings.json not found. Loading defaults
Deck calibration not found.
/Users/lucaslevassor/.opentrons/deck_calibration.json not found. Loading defaults


FileNotFoundError: Unable to find a labware
        definition for "96_wellplate_200ul",
        version 1, in the opentrons namespace.
        Please confirm your protocol includes the correct
        labware spelling and (optionally) the correct version
        number and namespace.

        If you are referencing a custom labware in your
        protocol, you must add it to your Custom Labware
        Definitions Folder from the Opentrons App before
        uploading your protocol.
        